## Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tweepy
import requests

#### Importing archive File

In [2]:
dogs_df = pd.read_csv('twitter-archive-enhanced.csv')
dogs_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
dogs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [4]:
dogs_df.tweet_id.nunique()

2356

#### Wrangling tweet image prediction image_predictions.tsv file from Udacity URL

In [5]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

with open('image_predictions.tsv', 'wb') as file:
    file.write(r.content)

In [6]:
predict_df = pd.read_csv('image_predictions.tsv', sep='\t')
predict_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### Wrangling twitter retweets and likes using twiter API

In [7]:
consumer_key = '0ngY2TCKTO0SqaysTtCRuCKkj'
consumer_secret = 'P6QFe66DdW3Xyw2dY1El0bPgaFkywQs4rgb9xGYSy7kK1pdF0z'

access_token = '695797976975990784-b6FSqvHysyaMln7lerN0tjxqbFnhOgm'
access_secret = '1ooTr0UBlNULWMIhOp7YIj4vITQVnVLw3xh5M6a2jyok4'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit= True, wait_on_rate_limit_notify  = True)

In [8]:
from pprint import pprint
import json
import os
import time

def dump_tweet_json(tweet_id, file):
    """
    query the tweet json file specified by twitter id 'tweet_id'
    using twitter API tweepy, parse the json object and write it to the
    file 'file'
    
    Warning! file is assumed open in writing mode
    """
    tweet_status = api.get_status(tweet_id, tweet_mode='extended')
    json.dump(tweet_status._json, file)
    

In [9]:
deleted_tweets_ids = []
rate_limit_exceeded_tweets  = []

with open('tweet_json.txt', 'w') as file:
    
    ## First Path of Querying tweets
    ## collecting deleted tweets ids and rate_limit_exceeded_tweets
    file.write('[\n')
    for tweet_id in dogs_df.tweet_id:
        try:
            
            dump_tweet_json(tweet_id, file)
            file.write(',\n')
        
        except tweepy.error.RateLimitError:
            rate_limit_exceeded_tweets.append(tweet_id)
        
        except Exception as e:
            deleted_tweets_ids.append(tweet_id)
            print(e,' ', tweet_id)
    
    ## Second Path of Querying tweets
    ## Making sure to collect rate_limit_exceeded_tweets if any 
    
    while rate_limit_exceeded_tweets:
        try:
            tweet_id = rate_limit_exceeded_tweets[0]
            dump_tweet_json(tweet_id, file)
            file.write(',\n')
            rate_limit_exceeded_tweets.pop(0)
            
        except tweepy.error.RateLimitError:
            time.sleep(60)
            
        except tweepy.error.TweepError as tr:
            if tr.message[0]['code'] == 144:
                rate_limit_exceeded_tweets.pop(0)
            else:
                print('Twitter Error: ', twitter_err)
        
        except Exception as e:
            print(e)
            
    file.seek(file.tell() - 3, os.SEEK_SET)
    file.write('\n]')

[{'code': 144, 'message': 'No status found with that ID.'}]   888202515573088257
[{'code': 144, 'message': 'No status found with that ID.'}]   873697596434513921
[{'code': 144, 'message': 'No status found with that ID.'}]   872668790621863937
[{'code': 144, 'message': 'No status found with that ID.'}]   872261713294495745
[{'code': 144, 'message': 'No status found with that ID.'}]   869988702071779329
[{'code': 144, 'message': 'No status found with that ID.'}]   866816280283807744
[{'code': 144, 'message': 'No status found with that ID.'}]   861769973181624320
[{'code': 144, 'message': 'No status found with that ID.'}]   856602993587888130
[{'code': 144, 'message': 'No status found with that ID.'}]   845459076796616705
[{'code': 144, 'message': 'No status found with that ID.'}]   844704788403113984
[{'code': 144, 'message': 'No status found with that ID.'}]   842892208864923648
[{'code': 144, 'message': 'No status found with that ID.'}]   837012587749474308
[{'code': 144, 'message': 'N

Rate limit reached. Sleeping for: 309


[{'code': 144, 'message': 'No status found with that ID.'}]   754011816964026368
[{'code': 144, 'message': 'No status found with that ID.'}]   680055455951884288


Rate limit reached. Sleeping for: 320


In [44]:
## Reading needed information from tweet_json to a dataframe
## needed info.: retweet_count, favorite_count
## in_reply_to_status_id, in_reply_to_user_id

df = pd.read_json('tweet_json.txt')
df.head(3)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,2017-08-01 16:23:56,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...",37554,False,This is Phineas. He's a mystical boy. Only eve...,NaN,...,NaN,NaN,NaN,NaN,8193,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
1,NaN,NaN,2017-08-01 00:17:27,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892177413194625024, 'id_str'...",32278,False,This is Tilly. She's just checking pup on you....,NaN,...,NaN,NaN,NaN,NaN,6060,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."
2,NaN,NaN,2017-07-31 00:18:03,"[0, 121]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 891815175371796480, 'id_str'...",24320,False,This is Archie. He is a rare Norwegian Pouncin...,NaN,...,NaN,NaN,NaN,NaN,4008,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


In [45]:
df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'favorite_count', 'favorited',
       'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'lang', 'place', 'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status_permalink', 'retweet_count', 'retweeted',
       'retweeted_status', 'source', 'truncated', 'user'],
      dtype='object')

In [46]:
## Checking completeness of wrangled parsed data and given data 
## After parsing the dataframe obs should be smaller than the given original data obs
## by just the deleted tweets

assert dogs_df.shape[0] == len(deleted_tweets_ids) + df.shape[0]

In [47]:
## Interested tweets info 

tweets_add_df = df[['retweet_count' , 'favorite_count', 'in_reply_to_status_id', 
                    'in_reply_to_user_id', 'is_quote_status', 'quoted_status_id', 'retweeted_status']]

tweets_add_df.head(3)

,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,quoted_status_id,retweeted_status
0,8193,37554,NaN,NaN,False,NaN,NaN
1,6060,32278,NaN,NaN,False,NaN,NaN
2,4008,24320,NaN,NaN,False,NaN,NaN


In [48]:
list(df['user'].apply(pd.Series))

['id',
 'id_str',
 'name',
 'screen_name',
 'location',
 'description',
 'url',
 'entities',
 'protected',
 'followers_count',
 'friends_count',
 'listed_count',
 'created_at',
 'favourites_count',
 'utc_offset',
 'time_zone',
 'geo_enabled',
 'verified',
 'statuses_count',
 'lang',
 'contributors_enabled',
 'is_translator',
 'is_translation_enabled',
 'profile_background_color',
 'profile_background_image_url',
 'profile_background_image_url_https',
 'profile_background_tile',
 'profile_image_url',
 'profile_image_url_https',
 'profile_banner_url',
 'profile_link_color',
 'profile_sidebar_border_color',
 'profile_sidebar_fill_color',
 'profile_text_color',
 'profile_use_background_image',
 'has_extended_profile',
 'default_profile',
 'default_profile_image',
 'following',
 'follow_request_sent',
 'notifications',
 'translator_type']

In [50]:
## Collecting info about users posting the tweets, most of them expected to be WeRateDogs page
## but if the tweet is a retweet, Different users shall be expected 

users_info = df['user'].apply(pd.Series)[['id', 'name', 'created_at', 'friends_count', 'followers_count', 'favourites_count', 'listed_count', 'statuses_count']]
users_info.head()

,id,name,created_at,friends_count,followers_count,favourites_count,listed_count,statuses_count
0,4196983835,WeRateDogs™,Sun Nov 15 21:41:29 +0000 2015,12,8026597,141917,6187,10170
1,4196983835,WeRateDogs™,Sun Nov 15 21:41:29 +0000 2015,12,8026597,141917,6187,10170
2,4196983835,WeRateDogs™,Sun Nov 15 21:41:29 +0000 2015,12,8026597,141917,6187,10170
3,4196983835,WeRateDogs™,Sun Nov 15 21:41:29 +0000 2015,12,8026597,141917,6187,10170
4,4196983835,WeRateDogs™,Sun Nov 15 21:41:29 +0000 2015,12,8026597,141917,6187,10170


<hr>

## Assessing and Defining Issues in the Data

In [114]:
dogs_df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None


In [115]:
tweets_add_df.head()

,retweet_count,favorite_count,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,quoted_status_id,retweeted_status
0,8193,37554,NaN,NaN,False,NaN,NaN
1,6060,32278,NaN,NaN,False,NaN,NaN
2,4008,24320,NaN,NaN,False,NaN,NaN
3,8347,40905,NaN,NaN,False,NaN,NaN
4,9040,39097,NaN,NaN,False,NaN,NaN


In [116]:
predict_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### Assessing dogs_df

> In this section, I will assess the data. In every step, I may find an issue and will document it in markdown cells, 
at the end of the section, all issues will be summarized and classified.

In [52]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [53]:
dogs_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


In [54]:
dogs_df.tail()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a 1949 1st generation vulpix. Enjoys sweat tea and Fox News. Cannot be phased. 5/10 https://t.co/4B7cOc1EDq,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248165822465/photo/1,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a purebred Piers Morgan. Loves to Netflix and chill. Always looks like he forgot to unplug the iron. 6/10 https://t.co/DWnyCjf2mx,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226329800704/photo/1,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here is a very happy pup. Big fan of well-maintained decks. Just look at that tongue. 9/10 would cuddle af https://t.co/y671yMhoiR,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412701032449/photo/1,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a western brown Mitsubishi terrier. Upset about leaf. Actually 2 dogs here. 7/10 would walk the shit out of https://t.co/r7mOb2m0UI,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285002620928/photo/1,7,10,a,None,None,None,None
2355,666020888022790149,NaN,NaN,2015-11-15 22:32:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666020888022790149/photo/1,8,10,None,None,None,None,None


In [56]:
dogs_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
423,821765923262631936,NaN,NaN,2017-01-18 17:07:18 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Duchess. She uses dark doggo forces to levitate her toys. 13/10 magical af https://t.co/maDNMETA52,NaN,NaN,NaN,https://twitter.com/dog_rates/status/821765923262631936/photo/1,13,10,Duchess,doggo,None,None,None
1157,724405726123311104,NaN,NaN,2016-04-25 01:12:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Ashleigh. She's having Coachella withdrawals. Didn't even go tho. 10/10 stay strong pupper https://t.co/nRUaKWnJfH,NaN,NaN,NaN,https://twitter.com/dog_rates/status/724405726123311104/photo/1,10,10,Ashleigh,None,None,pupper,None
343,832040443403784192,NaN,NaN,2017-02-16 01:34:34 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Klein. These pics were taken a month apart. He knows he's a stud now. 12/10 total heartthrob https://t.co/guDkLrX8zV,7.699404e+17,4.196984e+09,2016-08-28 16:51:16 +0000,"https://twitter.com/dog_rates/status/769940425801170949/photo/1,https://twitter.com/dog_rates/status/769940425801170949/photo/1,https://twitter.com/dog_rates/status/769940425801170949/photo/1,https://twitter.com/dog_rates/status/769940425801170949/photo/1",12,10,Klein,None,None,None,None
650,792883833364439040,NaN,NaN,2016-10-31 00:20:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bailey. She's rather h*ckin hype for Halloween tomorrow. Carved those pupkins herself. 12/10 https://t.co/v17mFm0Ftz,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/792883833364439040/photo/1,https://twitter.com/dog_rates/status/792883833364439040/photo/1,https://twitter.com/dog_rates/status/792883833364439040/photo/1,https://twitter.com/dog_rates/status/792883833364439040/photo/1",12,10,Bailey,None,None,None,None
1772,678278586130948096,NaN,NaN,2015-12-19 18:19:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Another spooky pupper here. Most definitely floating. No legs. Probably knows some dark magic. 10/10 very spooked https://t.co/JK8MByRzgj,NaN,NaN,NaN,https://twitter.com/dog_rates/status/678278586130948096/photo/1,10,10,None,None,None,pupper,None


In [57]:
dogs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

From visual assessment and info() function:
* the source column contains an html link tag that contians three types of information in one column, the source of the tweet, link of the source, 'rel' attribute.
* The doggo, pupper and puppo are all possible dog stage, so it's better  of to represent them by one column e.g dog_stage column and the stage_column should be categorized.
* Column doggo, pupper, and puppo sometimes are all Nones while a dog should be really in one of the three stages or sometimes big doggo is considered woofer, but here in the system, it seems dogs are categorized between only pupper, doggo and puppo and from further investigation visual assessment, it seems that the category of the dog is extracted from the text, so when the tweet text doesn't contain any of these words or use different abbreviations like 'pup', the values are none.
* Not all ratings numerators are higher than 10, some ratings actually below 10, from visual assessment, and seeing the dates, it seems all in this small sample posted in 2015, more programmatic assessment needed here.
* The (in_reply_to_status_id, in_reply_to_user_id), (retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp)	seems to contain many Nones, as we see from the info function, just 78 non-nulls for in_reply and 181 for retweets and that might not come as a surprise since most tweets are originally posted by the page, and such official rating pages may not retweet often, rather a personal twitter may contain many retweets.
* The doggo, floofer, pupper, puppo doesn't contain any null entries from the info() function although we seem many Nones from the visual assessment and could be that string 'None' is inserted in them not object None.
* expanded_urls contains 59 null objects that means there are 59 tweets with no url, this data could be gathered from the json file, if exists.
* There is a 'None' names in the tail and 'a' names, too. Checking the text, it seems names are extracted from statements in the form 'This is name', 'Here is name', if anything came in place of name, it's would be the name, and if the pattern doesn' exist, it will be None. A better text pattern may be used, but from this sample head, tail, sample, Sometimes names aren't really stated, somtimes breed, type or origin is stated, so we should furhter investigate name column.
* 'tweet_id' is integer, 'in_reply_to_status_id', 'retweeted_status_user_id', 'retweeted_status_id', 'retweeted_status_user_id' are floats. Finally, timestamp columns are string objects not datetime.

Next I will check for values of every interesting column to make sure case of 'None' strings doesn't happen to other columns

In [60]:
def value_counts_per_column(df, columns):
    """
    Takes a dataframe and a set of columns in the dataframe 
    and prints out the value_counts of each column
    """
    for column in columns:
        print(df[column].value_counts())
        print('\n\n', '==' * 20, '\n\n')
        

In [61]:
list(dogs_df)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo']

In [63]:
value_counts_per_column(dogs_df, ['name', 'doggo', 'floofer', 'pupper', 'puppo', 'source'])

None         745
a            55 
Charlie      12 
Lucy         11 
Oliver       11 
Cooper       11 
Tucker       10 
Lola         10 
Penny        10 
Winston      9  
Bo           9  
the          8  
Sadie        8  
Bailey       7  
Daisy        7  
Buddy        7  
an           7  
Toby         7  
Koda         6  
Rusty        6  
Oscar        6  
Scout        6  
Stanley      6  
Jack         6  
Jax          6  
Dave         6  
Milo         6  
Bella        6  
Leo          6  
Larry        5  
            ..  
Spark        1  
Binky        1  
Tyrus        1  
Tripp        1  
Ferg         1  
Link         1  
Dwight       1  
Acro         1  
Eleanor      1  
Storkson     1  
Charleson    1  
Mason        1  
Bert         1  
Andru        1  
Aqua         1  
Combo        1  
Maxwell      1  
Beckham      1  
Zeek         1  
Zeus         1  
Hanz         1  
Newt         1  
Zooey        1  
Flash        1  
Fabio        1  
Godzilla     1  
Einstein     1  
old          1

**Observations per column(s):**
* 'name': There are many Nones and other unreasonable names like 'a', 'an', 'old', 'the'
* 'doggo', 'floofer', 'pupper', 'puppo': Values are either None or their column name, 'None' are dominant in those columns. 
* 'source': is either from Twitter for iPhone app, Vine, the Web or TweetDeck, we can see that the rel attribute takes always 'nofollow' value, so actually its existence when tidying up the column may be irrelevant, more space for no benefit, we make make a note that all links are 'nofollow' for browsers. Hence, I may argue that the html attr 'rel' is not important in our investigation or representation of the data-set.

In [73]:
## Further investigating name column

pd.set_option('display.max_rows', 500)

dogs_df['name'].value_counts()[dogs_df['name'].value_counts() > 3]

None        745
a           55 
Charlie     12 
Lucy        11 
Oliver      11 
Cooper      11 
Tucker      10 
Lola        10 
Penny       10 
Winston     9  
Bo          9  
the         8  
Sadie       8  
Bailey      7  
Daisy       7  
Buddy       7  
an          7  
Toby        7  
Koda        6  
Rusty       6  
Oscar       6  
Scout       6  
Stanley     6  
Jack        6  
Jax         6  
Dave        6  
Milo        6  
Bella       6  
Leo         6  
Larry       5  
Gus         5  
very        5  
Alfie       5  
Louis       5  
Sammy       5  
George      5  
Phil        5  
Finn        5  
Bentley     5  
Sunny       5  
Chester     5  
Oakley      5  
Boomer      4  
Reggie      4  
Clarence    4  
Archie      4  
Clark       4  
Loki        4  
Luna        4  
Hank        4  
Winnie      4  
Maddie      4  
one         4  
Shadow      4  
Walter      4  
Maggie      4  
just        4  
Ruby        4  
quite       4  
Sampson     4  
Bruce       4  
Dexter      4  
Gerald  

<hr>

**Investigating name column furhter we notice those issues:**
1. very high count of None (745)
2. Articles like 'a', 'an', 'the' adverbs like 'just', 'very', 'quite', adj like 'old' and others like  'one'.
3. Some Strange names like Chip Scooter Moose Bentley Shadow, but names by nature aren't justified, so I will dig further to those specific names, but most likely they won't considered quality issues.

The articles, adverbs and others like 'one' are all begin with lower case letters, that might help cleaning those data instead of getting list of all possible articles, adverbs and pronouns to check their existence in the names column

<hr>

In [110]:
def any_None_empty_object(df):
    """
    checks there is any 'None' or empty '' string objects values in string 
    columns and prints out the column name if any
    """
    for column in list(df):
        if df[column].dtype == np.object:
            if any(df[column].str.lower() == 'none'):
                print(column, "'none' strings exists")
            elif any(df[column].str.strip() == ''):
                print(column, 'Empty strings exist')

In [111]:
any_None_empty_object(dogs_df)

name 'none' strings exists
doggo 'none' strings exists
floofer 'none' strings exists
pupper 'none' strings exists
puppo 'none' strings exists


<hr>

It seems there is no empty string objects for any string column and srings containing 'none' strings are name, doggo, pupper, puppo which we have assessed before.

<hr>

In [95]:
pd.set_option('display.max_rows', 10)

In [109]:
## Additional assessment for remaining text columns

value_counts_per_column(dogs_df, ['timestamp', 'text', 'expanded_urls', 'retweeted_status_timestamp'])

2016-03-05 04:17:02 +0000    1
2015-12-07 02:21:29 +0000    1
2015-11-19 03:52:34 +0000    1
2015-12-03 02:21:48 +0000    1
2017-01-22 01:12:59 +0000    1
                            ..
2016-02-04 02:00:27 +0000    1
2016-03-10 01:35:01 +0000    1
2016-02-06 03:50:33 +0000    1
2016-02-25 16:53:11 +0000    1
2015-11-28 19:04:19 +0000    1
Name: timestamp, Length: 2356, dtype: int64




This is Orion. He just got back from the dentist. Cavity free af. 12/10 would give extra pats https://t.co/Y4DZx2UWsr                           1
This is Ike. He's demonstrating the pupmost restraint. 13/10 super good boy https://t.co/6gHoGah9nm                                             1
This is Daisy. She's puppears to be rare as all h*ck. Only seven like her currently domesticated. 13/10 pettable af https://t.co/meUc8jufAO     1
"Don't ever talk to me or my son again." ...both 10/10 https://t.co/b8ncwl6TlE                                                                  1
Meet Rilo. He's a Northern 

**Observations:**
* Timestamp objects seems to be unique and all follow same format.
* Texts also tend to be unique as there is no text with value_count more than 1
* Expaned urls have some urls with value count more than one while we know every tweet has unique tweet_id. 
<br>In addition, some expanded urls have their url repeated multiple times like "https://twitter.com/dog_rates/status/782305867769217024/photo/1,
https://twitter.com/dog_rates/status/782305867769217024/photo/1,
https://twitter.com/dog_rates/status/782305867769217024/photo/1" This tend to be a quality issue to be considered in the cleaning.

<hr>

In [112]:
## Checking uniqueness of tweet_id and that there is no None values
assert dogs_df['tweet_id'].nunique() == dogs_df.shape[0]

In [74]:
dogs_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


**Observations:**
* Checking (min, 25%, 50%, 75%, max), every tweet id (tweet_id, in_reply_to_status_id, retweeted_status_id) tend to be 18 digits, hence a string representation here is meaningful as mean, std doens't really mean anything here.
* On the other hand, user ids (in_reply_to_user_id, retweeted_status_user_id) doesn't all have 18 digits, some have 8 or 6 digits, up to 18 digits, according to the article [Interpreting Twitter id](https://www.adweek.com/digital/twitter-user-id/) twitter user ids gave an ordering of when a user account was created, but thinking of the 18 digit user ids, it seems the incrementing or incrementing fashion like user id system has been replaced by 18 digits, that's just a guess, to make sure we should investigate user ids and their account launch dates. In addition, this guess is based on the fact that there is supposedly about 330 million twitter user today and there is about 7.5 billion people in the world as whole, 18 digits is much much higher number, meaning even if every person on Earth is registered on twitter, every one shall have about a billion twitter accounts which is untrue.
* Rating numerators have untrue or outlier values like 1776, 0 biasing the mean and std. (quality issue)
* Rating denomenators have untrue or outlier values like 0, 170 biasing while all denomenators should be supposedly 10. (quality issue)

> **Summarizing Data issues in dogs_df:**
* **Tidyness Issues: **
       1. Converting source column HTML to its constituent parts url, and app source, 'rel' will be discarded since it's alwasy 'no-follow'.
       2. Converting pupper, puppo, doggo value columns to be values in one column stage_column.
* **Quality Issues: **
       1. 'name' column: Clearing 'None' names and inappropriate false names like 'a', 'an' and 'the'.
       2. Changing 'floofer' column to boolean.

### Cleaning dogs_df

In [118]:
dogs_df.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None


<hr>

fixing puppo, pupper, doggo columns

In [135]:
## Checking for crossed values before melting

# this function checks if there dogs classified as either (doggo, pupper), 
# (doggo, puppo), (pupper, puppo) or (doggo, pupper, puppo)

dogs_stages = ['doggo', 'puppo', 'pupper']

for i in range(len(dogs_stages)):
    j = i + 1
    for j in range(i + 1, len(dogs_stages)):
        print('(%s, %s)' %(dogs_stages[i], dogs_stages[j]),
             dogs_df.query('%s == "%s" and %s == "%s"'%(dogs_stages[i], dogs_stages[i], dogs_stages[j], dogs_stages[j])).shape[0]
             )

## Checking for three columns
print('(%s, %s, %s)' %('doggo', 'puppo', 'pupper'),
             dogs_df.query('doggo == "doggo" and puppo == "puppo" and pupper == "pupper"').shape[0]
     )

(doggo, puppo) 1
(doggo, pupper) 12
(puppo, pupper) 0
(doggo, puppo, pupper) 0


In [ ]:
cross_doggo_pupper_puppo_check

In [126]:
dogs_df.query('puppo == "puppo" and pupper == "pupper"').shape

(0, 17)

In [ ]:
pd.melt(id_vars=['tweet_id'], value_vars=['doggo', 'puppo', 'pupper'])